# Day 3 - Function Calling with the Gemini API

Welcome to the Generative AI Course!

In this notebook, you will use the Gemini API's automatic function calling to build a chat interface over a local database. This allows the model to "use" your code to answer questions that require current or private data.

**Prerequisites**:
- A Google Cloud Project with the Gemini API enabled.
- An API key stored in the `GOOGLE_API_KEY` environment variable.

## Setup

```bash
pip install -U -q "google-genai"
```

In [ ]:
import os
import sqlite3
from google import genai
from google.genai import types

### Set up your API key

In [ ]:
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("Please set the GOOGLE_API_KEY environment variable.")

client = genai.Client(api_key=GOOGLE_API_KEY)

## Create a local database

For this example, you'll create a local SQLite database and add some synthetic data about a computer store.

In [ ]:
DB_FILE = "sample.db"
db_conn = sqlite3.connect(DB_FILE)

# Reset tables if they exist
cursor = db_conn.cursor()
cursor.execute("DROP TABLE IF EXISTS orders")
cursor.execute("DROP TABLE IF EXISTS products")
cursor.execute("DROP TABLE IF EXISTS staff")

# Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name VARCHAR(255) NOT NULL,
    price DECIMAL(10, 2) NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS staff (
    staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name VARCHAR(255) NOT NULL,
    staff_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
)
""")

# Insert data
cursor.execute("INSERT INTO products (product_name, price) VALUES ('Laptop', 799.99), ('Keyboard', 129.99), ('Mouse', 29.99)")
cursor.execute("INSERT INTO staff (first_name, last_name) VALUES ('Alice', 'Smith'), ('Bob', 'Johnson'), ('Charlie', 'Williams')")
cursor.execute("INSERT INTO orders (customer_name, staff_id, product_id) VALUES ('David Lee', 1, 1), ('Emily Chen', 2, 2), ('Frank Brown', 1, 3)")

db_conn.commit()
print("Database created and populated.")

## Define database functions

You will define three Python functions that the model can use to interact with the database:
1. `list_tables()`: To adapt to any database structure.
2. `describe_table(table_name)`: To understand the schema.
3. `execute_query(sql)`: To run the SQL generated by the model.

**Crucial**: Provide clear docstrings and type hints. The Gemini API uses these to construct the schema that is passed to the model.

In [ ]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    print(' - DB CALL: list_tables()')
    cursor = db_conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [t[0] for t in cursor.fetchall()]

def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.
    
    Args:
        table_name: The name of the table to describe.
        
    Returns:
        List of columns, where each entry is a tuple of (column_name, column_type).
    """
    print(f' - DB CALL: describe_table({table_name})')
    cursor = db_conn.cursor()
    # Safely executing PRAGMA with f-string is generally okay for local trusted tools, 
    # but be wary of SQL injection in production with arbitrary user input.
    cursor.execute(f"PRAGMA table_info({table_name});")
    # schema: [cid, name, type, notnull, dflt_value, pk]
    return [(col[1], col[2]) for col in cursor.fetchall()]

def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL SELECT statement and return the results.
    
    Args:
        sql: The SQL query to execute.
        
    Returns:
        A list of rows, where each row is a list of string values.
    """
    print(f' - DB CALL: execute_query({sql})')
    cursor = db_conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()

# Test one function
print("Available tables:", list_tables())

## Configure the tools

You simply pass the Python functions into the `tools` list. The `google-genai` SDK processes them automatically.

In [ ]:
db_tools = [list_tables, describe_table, execute_query]

# System instruction to guide the model's behavior
# It gives the model a persona and specific instructions on how to use the tools.
instruction = """You are a helpful chatbot that can interact with an SQL database.
You will take the user's questions and turn them into SQL queries using the tools available.
Once you have the information you need, you will answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the schema,
and execute_query to issue an SQL SELECT query."""

## Chat with the Data

We will use `client.chats.create` to start a session with automatic function calling enabled.

In [ ]:
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
        # 'automatic_function_calling' is enabled by default when tools are provided in chats
    ),
)

Ask a question that requires multiple steps (listing tables, seeing schema, querying data).

In [ ]:
response = chat.send_message("What is the cheapest product?")
print(f"\nModel Answer:\n{response.text}")

Ask a follow-up question. The model remembers the context (table structures) from the previous turn.

In [ ]:
response = chat.send_message("Who sold it?")
print(f"\nModel Answer:\n{response.text}")

## Inspecting the Conversation

You can view the history to see the back-and-forth between the model and the tools.

In [ ]:
for event in chat.get_history():
    print(f"\nRole: {event.role}")
    for part in event.parts:
        if part.text:
            print(f"Text: {part.text}")
        elif part.function_call:
            print(f"Function Call: {part.function_call.name}({part.function_call.args})")
        elif part.function_response:
            print(f"Function Response: {part.function_response.response}")

## Further reading

To learn more about what the Gemini API can do with function calling, check out the [Function calling cookbook](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb) (see `Manual function calling` to understand how function calling works manually) as well as [Function calling config](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb), which gives you fine-grained control over how function calling is triggered.

And stay tuned for day 4, where you will explore using function calling with grounding tools.